In [1]:

from google.colab import drive
drive.mount('/content/drive/')

from zipfile import ZipFile
file_name='/content/drive/MyDrive/severstal-steel-defect-detection.zip'
with ZipFile(file_name,'r') as Zip:
  Zip.extractall()

Mounted at /content/drive/


# Yeni Bölüm

In [2]:
!pip install -U tensorflow-addons

import pandas as pd
import numpy as np
import cv2
import time
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sb
import os
import math
from random import random
import tensorflow as tf
import keras
from PIL import Image
from tensorflow.keras import backend as K
from keras.layers import Input, Lambda, Flatten,Conv2D,Dense,MaxPooling2D,Dropout,BatchNormalization,GlobalAveragePooling2D,Activation,Add,UpSampling2D,Concatenate,LeakyReLU,Conv2DTranspose,concatenate
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard
import tensorflow_addons as tfa
import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Layer
from tensorflow.keras.applications import EfficientNetB0

%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.0 MB/s 


File


In [3]:
df=pd.read_csv('train.csv')
print(df.shape)
df.head(2)

(7095, 3)


,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...


In [4]:
#https://stackoverflow.com/questions/58693261/create-a-rle-run-lenth-encoding-mask-with-tensorflow-datasets
def rle2mask(mask_rle):
  shape = tf.convert_to_tensor((1600,256), tf.int64)
  size = tf.math.reduce_prod(shape)
  s = tf.strings.split(mask_rle)
  s = tf.strings.to_number(s, tf.int64)
  starts = s[::2] - 1
  lens = s[1::2]
  total_ones = tf.reduce_sum(lens)
  ones = tf.ones([total_ones], tf.uint8)
  r = tf.range(total_ones)
  lens_cum = tf.math.cumsum(lens)
  s = tf.searchsorted(lens_cum, r, 'right')
  idx = r + tf.gather(starts - tf.pad(lens_cum[:-1], [(1, 0)]), s)
  mask_flat = tf.scatter_nd(tf.expand_dims(idx, 1), ones, [size])
  mask  = tf.transpose(tf.reshape(mask_flat, shape)[::2,::])
  return mask

Dice Coef and Loss
0.0000000001

In [5]:
smooth = 1e-10
def dice_coef(y_true, y_pred):
    y_true =  tf.cast(y_true,'float32')
    y_pred =  tf.cast(y_pred,'float32')
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)    